In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.simplefilter('ignore')
# Ignore only the specific FutureWarning from pandas option
warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    message=r".*use_inf_as_na option is deprecated.*"
)

# Setting matplotlib defaults
plt.rc('figure', figsize=(8, 5), dpi=120)

plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/black-jack-smart-effect-of-removal-ml/sample_submission.csv
/kaggle/input/black-jack-smart-effect-of-removal-ml/train.csv
/kaggle/input/black-jack-smart-effect-of-removal-ml/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/black-jack-smart-effect-of-removal-ml/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/black-jack-smart-effect-of-removal-ml/test.csv', index_col='id')

# Data Understanding

In [3]:
train.sample(5)

,1,2,3,4,5,6,7,8,9,10,ev
id,,,,,,,,,,,
208,14,12,6,11,8,13,14,13,11,38,-0.029709
6463,17,20,17,12,16,17,17,13,14,57,-0.008156
253,7,7,11,11,13,6,9,10,12,34,-0.010651
18164,1,1,2,2,1,2,1,1,0,9,-0.025465
6315,3,2,2,3,6,2,1,3,4,14,-0.024081


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20500 entries, 0 to 20499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       20500 non-null  int64  
 1   2       20500 non-null  int64  
 2   3       20500 non-null  int64  
 3   4       20500 non-null  int64  
 4   5       20500 non-null  int64  
 5   6       20500 non-null  int64  
 6   7       20500 non-null  int64  
 7   8       20500 non-null  int64  
 8   9       20500 non-null  int64  
 9   10      20500 non-null  int64  
 10  ev      20500 non-null  float64
dtypes: float64(1), int64(10)
memory usage: 1.9 MB


In [5]:
num_cols = train.select_dtypes(include=['int64']).columns
train[num_cols] = train[num_cols].astype('int8')
test[num_cols] = test[num_cols].astype('int8')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20500 entries, 0 to 20499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       20500 non-null  int8   
 1   2       20500 non-null  int8   
 2   3       20500 non-null  int8   
 3   4       20500 non-null  int8   
 4   5       20500 non-null  int8   
 5   6       20500 non-null  int8   
 6   7       20500 non-null  int8   
 7   8       20500 non-null  int8   
 8   9       20500 non-null  int8   
 9   10      20500 non-null  int8   
 10  ev      20500 non-null  float64
dtypes: float64(1), int8(10)
memory usage: 520.5 KB


In [7]:
train.shape

(20500, 11)

In [8]:
test.shape

(8655, 10)

In [9]:
train.isnull().sum()

1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
ev    0
dtype: int64

In [10]:
test.isnull().sum()

1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

In [11]:
train.describe()

,1,2,3,4,5,6,7,8,9,10,ev
count,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000
mean,7.591756,7.601073,7.597951,7.577659,7.598439,7.592683,7.552049,7.568878,7.602439,30.334634,-0.023383
std,4.809277,4.771329,4.799255,4.764657,4.790892,4.796563,4.754059,4.773905,4.766625,17.658349,0.010041
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.071232
25%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,15.000000,-0.029104
50%,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,28.000000,-0.024109
75%,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,44.000000,-0.018508
max,22.000000,22.000000,22.000000,21.000000,22.000000,22.000000,22.000000,22.000000,22.000000,74.000000,0.044198


In [12]:
test.describe()

,1,2,3,4,5,6,7,8,9,10
count,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000
mean,7.622415,7.614211,7.564298,7.553206,7.574466,7.541075,7.574003,7.549856,7.605893,30.395609
std,4.770199,4.773575,4.765060,4.773822,4.785457,4.737471,4.761329,4.762710,4.804795,17.683448
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,15.000000
50%,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,28.000000
75%,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,44.000000
max,22.000000,22.000000,22.000000,22.000000,23.000000,22.000000,21.000000,21.000000,23.000000,73.000000


In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

X = train.copy()
y = X.pop('ev')
X_test = test.copy()

n_folds=10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=34)

oof_dnn = np.zeros(len(y))
test_preds_dnn = np.zeros((len(X_test), n_folds))
fold_mse_dnn = []


In [14]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

2025-04-30 07:59:18.264340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745999958.557953      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745999958.643554      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
oof_pred_dnn = np.zeros(len(X))
test_preds_dnn = np.zeros((len(X_test), n_folds))
fold_mse_dnn = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test) 

    def build_model():
        return Sequential([
            Input(shape=(X_train_scaled.shape[1],)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='leaky_relu'),
            Dense(1)
        ])

    val_preds, test_preds = [], []
    for rep in range(3):
        np.random.seed(34 + rep)
        tf.random.set_seed(34 + rep)
        
        model = build_model()
        model.compile(optimizer='adam', loss='mse')
        
        model.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=20,
            batch_size=32,
            callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
            verbose=0
        )
        
        val_preds.append(model.predict(X_val_scaled).flatten())
        test_preds.append(model.predict(X_test_scaled).flatten()) 

    fold_val_pred = np.mean(val_preds, axis=0)
    fold_test_pred = np.mean(test_preds, axis=0)
    
    fold_mse = mean_squared_error(y_val, fold_val_pred)
    fold_mse_dnn.append(fold_mse)
    oof_pred_dnn[val_idx] = fold_val_pred
    test_preds_dnn[:, fold-1] = fold_test_pred
    
    print(f"Fold {fold} MSE: {fold_mse:.9f}")

final_test_pred_dnn = test_preds_dnn.mean(axis=1)
print(f"\nOverall OOF MSE: {mean_squared_error(y, oof_pred_dnn):.8f}")
print("Test predictions (first 10):", final_test_pred_dnn[:10])

2025-04-30 07:59:34.390479: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold 1 MSE: 0.000000490
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold 2 MSE: 0.000000704
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Fold 3 MSE: 0.000000654
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
271/271 ━

In [16]:
sub = pd.read_csv('/kaggle/input/black-jack-smart-effect-of-removal-ml/sample_submission.csv')
sub['ev'] = final_test_pred_dnn 
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
